In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 19.7 MB/s eta 0:00:00


In [ ]:
!pip install pyvi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 41.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 1.7 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
import numpy as np
from datasets import load_metric
from huggingface_hub import notebook_login
from transformers import TrainingArguments, Trainer
from pyvi import ViTokenizer

 

In [ ]:
#ncduy/phobert-large-finetuned-vietnamese_students_feedback
classifier = pipeline("sentiment-analysis", model ="ncduy/phobert-large-finetuned-vietnamese_students_feedback")

In [ ]:
#load the fixed data
train_data= [ ]

with open("/content/drive/MyDrive/TEMPORUN/corrected_data.txt", "r") as myfile: 
    lines = myfile.read().splitlines()
    
    for line in lines: 
        pack={}
        text = line.split("\t")
        print(text)
        pack['text']= text[1].lower()
        #0 : positive , 1: negative , 2 neutral 
        if text[0]== "positive": 
          pack['label']= 0 
        elif text[0]== "negative": 
          pack['label']= 1 
        elif text[0]== "neutral": 
          pack['label']= 2 
     
    
        train_data.append(pack)
    myfile.close() 
len(train_data)

In [ ]:


test_data= [ ]

with open("/content/drive/MyDrive/TEMPORUN/data_phase_0.txt", "r") as myfile: 
    lines = myfile.read().splitlines()
    
    for line in lines:
      pack={} 
      pack["text"]= line
      test_data.append(pack)
    myfile.close() 
len(test_data)

100

### preprocessing


In [ ]:
#vietnamese corrector 
#use the corrector if there are some mis spelling in the data 
from transformers import pipeline

corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction")

In [ ]:
!pip install translate 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#translating back and forward 
from translate import Translator
translatorVIEN = Translator(to_lang="vi", from_lang="en")
translatorENVI = Translator(to_lang="en", from_lang="vi")

for i in range(5000): 
  print(i)
  if train_data[i]["label"] == 2: 
# Translate from Vietnamese to English
    translation = translatorVIEN.translate(train_data[i]["text"])
# Translate from English to Vietnamese
    translation = translatorENVI.translate(translation)
    pack={}
    pack["text"]= translation
    pack["label"]= 2
    train_data.append(pack)

 


In [ ]:
len(train_data)

6798

In [ ]:
countClass = [0,0,0]
for sample in train_data: 
  countClass[sample["label"]]+=1
countClass

[2361, 2245, 2192]

In [ ]:
#save the augmented data 
with open("/content/drive/MyDrive/TEMPORUN/new_data.txt", "w") as myfile: 
  for train in train_data: 
    if train["label"] == 0: 
        string = "positive\t"  + train["text"] + "\n"
        myfile.write(string)  
       
    elif train["label"] == 1: 
        string = "negative\t"  + train["text"] +"\n"

        myfile.write(string) 
    elif train["label"] == 2: 
        string = "neutral\t"+ "\t"  + train["text"] + "\n"

        myfile.write(string) 

  myfile.close()

In [ ]:
print("neutral\t" +"\t" + "hello")

neutral		hello


In [ ]:
#symnonyms replacement 
from pyvi import ViTokenizer, ViTokenizer, ViUtils,ViPosTagger
import requests
from bs4 import BeautifulSoup

def get_synonyms(word):
    VDict = ViUtils.load_dict()
    synonyms = VDict.get_synsets(word)
    return synonyms

def replace_synonyms(sentence):
    tokens = ViTokenizer.tokenize(sentence).split()
    tags=[]
    for token in tokens:  
        tags.append(ViPosTagger.postagging(token)) 
    result = []
    for i in range(len(tokens)):
        word = tokens[i]
        tag = tags[i]
        if tag in ['N', 'A', 'V']:
            synonyms = get_synonyms(word)
            if len(synonyms) > 1:
                synonyms.remove(word)
                new_word = synonyms[0]
            else:
                new_word = word
        else:
            new_word = word
        result.append(new_word)
        print(result)
    return ' '.join(result)

In [ ]:
for i in range(1000):
  i=i+1  
  if train_data[i]["label"]== 2: 
    pack={}
    new_line=  replace_synonyms(train_data[i]["text"])
    pack["text"]= new_line
    pack["label"]= 2
    train_data.append(pack)
  print("number :", i )
  i=i+1 


### fine-tuning 

In [ ]:
#load the fixed data
train_data= [ ]

with open("/content/drive/MyDrive/TEMPORUN/corrected_data.txt", "r") as myfile: 
    lines = myfile.read().splitlines()
    
    for line in lines: 
        pack={}
        text = line.split("\t")
        pack['text']= text[1].lower()
        #0 : positive , 1: negative , 2 neutral 
        if text[0]== "positive": 
          pack['label']= 0 
        elif text[0]== "negative": 
          pack['label']= 1 
        elif text[0]== "neutral": 
          pack['label']= 2 
     
    
        train_data.append(pack)
    myfile.close() 
len(train_data)

5000

In [ ]:
import torch

In [ ]:
# replace "number" base on how many data you want to use as train_test
temp = train_data
train_data= temp[:"number"]
test_data = temp["number": "number "]
f1_score_testing = temp["number ":]

In [ ]:
train_sentences = []
test_sentences= []
train_labels=[]
test_labels= []

for train in train_data: 
  train_sentences.append(train["text"])
  train_labels.append(train["label"])
for test in test_data: 
  test_sentences.append(test["text"]) 
  test_labels.append(test["label"])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import BertTokenizer,BertModel, RobertaTokenizer, RobertaForSequenceClassification,PhobertTokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = tokenizer = AutoTokenizer.from_pretrained("ncduy/phobert-large-finetuned-vietnamese_students_feedback")
model =  AutoModelForSequenceClassification.from_pretrained('ncduy/phobert-large-finetuned-vietnamese_students_feedback', num_labels=3)




In [ ]:
train_labels

In [ ]:
#tokenize the data and get some parameter to add tensorflow data loader 
#input_ids: the ids for each starting point of word vector 
#attention_masks : as we padding all the vector to the same size we only want the attention to stay close to point that represent the word's vector 
#labels_val : the label for the classes 
max_length = 128

encoded_data_train = tokenizer(train_sentences,
                               truncation=True,
                               padding=True,
                               max_length=max_length,
                               return_tensors='pt')

encoded_data_val = tokenizer(test_sentences,
                             truncation=True,
                             padding=True,
                             max_length=max_length,
                             return_tensors='pt')

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(train_labels)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(test_labels)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup

batch_size = 32 

train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
train_dataloader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)

validation_dataset = TensorDataset(input_ids_val, attention_masks_val, labels_val)
validation_dataloader = DataLoader(validation_dataset, sampler=SequentialSampler(validation_dataset), batch_size=batch_size)

epochs = 30

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)

        model.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs[0]
        logits = outputs[1]

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()

    print(f'Epoch {epoch + 1}/{epochs} - Training Loss: {loss.item()}')

    model.eval()

    eval_loss, eval_accuracy, eval_f1 = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            outputs = model(input_ids=batch[0], attention_mask=batch[1], labels=batch[2])

        loss = outputs[0]
        logits = outputs[1]

        eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = batch

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/30 - Training Loss: 0.6551679372787476
Epoch 2/30 - Training Loss: 0.5978164076805115
Epoch 3/30 - Training Loss: 0.4764469861984253
Epoch 4/30 - Training Loss: 0.38139402866363525
Epoch 5/30 - Training Loss: 0.5022857785224915
Epoch 6/30 - Training Loss: 0.29719987511634827
Epoch 7/30 - Training Loss: 0.05474419891834259
Epoch 8/30 - Training Loss: 0.09778375178575516
Epoch 9/30 - Training Loss: 0.006147883366793394
Epoch 10/30 - Training Loss: 0.015364040620625019
Epoch 11/30 - Training Loss: 0.013015675358474255
Epoch 12/30 - Training Loss: 0.22054293751716614
Epoch 13/30 - Training Loss: 0.02467469498515129
Epoch 14/30 - Training Loss: 0.0009867710759863257
Epoch 15/30 - Training Loss: 0.0004538640787359327
Epoch 16/30 - Training Loss: 0.0003055064007639885
Epoch 17/30 - Training Loss: 0.00015915147378109396
Epoch 18/30 - Training Loss: 0.0001527844142401591
Epoch 19/30 - Training Loss: 0.00012201574281789362
Epoch 20/30 - Training Loss: 0.028234772384166718
Epoch 21/30 - T

In [ ]:
%cd "the place you want to save the model"
!mkdir model

# save the model weights
model.save_pretrained('model')

/content/drive/MyDrive/TEMPORUN/result


In [ ]:
#take the trained weight and get the prediction 
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

# Tokenize input sentence
Link_to_model_weights = "/content/drive/MyDrive/TEMPORUN/result/model2"
def predict(sample, Link_to_model_weights):
  encoded_input = tokenizer.encode_plus(
      sample,
      max_length=128,
      truncation=True,
      padding='max_length',
      add_special_tokens=True,
      return_attention_mask=True,
      return_tensors='pt'
  )

  model = AutoModelForSequenceClassification.from_pretrained(Link_to_model_weights)
  #set model to evaluation mode 
  model.eval()


  # Get model prediction
  with torch.no_grad():
      outputs = model(**encoded_input)
      logits = outputs.logits
      predicted_class = torch.argmax(logits, dim=1).item()

  # Print the predicted class
  return predicted_class

In [ ]:
#get result file 
Link_to_weights= "/content/drive/MyDrive/TEMPORUN/result/model3"
with open("/content/drive/MyDrive/TEMPORUN/result/submit.txt", "w") as myfile: 
  i=0 
  for test in System_Testing: 
  
    prediction = predict(test, Link_to_model_weights)
    if prediction== 0: 
      myfile.write("positive"+ "\n")
    elif prediction== 1: 
      myfile.write("negative"+ "\n")
    elif prediction== 2: 
      myfile.write("neutral"+ "\n")
    print("number : ", i)
    i=i+1 
  myfile.close()



In [ ]:
Link_to_weights = "/content/drive/MyDrive/TEMPORUN/result/model3"
sentences="Thầy dạy rất hay"
y_pred = predict(sentences, Link_to_weights)
y_pred 


0

In [ ]:
#get f1_score 
import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score
Link_to_weights = "/content/drive/MyDrive/TEMPORUN/result/model2"
# assuming you have loaded your model and data, and have a variable `y_true` containing the true labels
y_true = test_labels
# predict the labels of the test data using the trained model
y_pred = []
for test in test_sentences: 
  y_pred.append(predict(test, Link_to_weights))

# convert the predicted probabilities into class labels (0 or 1 for binary classification, or 0, 1, 2, etc. for multi-class classification)




In [ ]:
#get the score 

precision = precision_score(y_pred=y_pred, y_true=y_true, average='macro')
recall = recall_score(y_pred=y_pred, y_true=y_true, average='macro')
f1 = f1_score(y_pred=y_pred, y_true=y_true, average='macro')

print("F1 score:", f1)